In [578]:
## importing

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
import json
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [2]:
test=pd.read_csv('shipping_data.csv', sep = ';')

In [3]:
test.fillna(0)

,#,Date,Order_ID,Order_date,Start_Shipping_Date,Estimated_Arrival_Date,Actual_Arrival_Date,Days_from_order_to_shipment,Days_from_shipment_to_delivery,Days_from_order_to_delivery,Days_between_estimated_and_actual_arrival,Product_Category,Mode_Of_Transport,Priority,Carrier_name,Warehouse,Supplier_Name,Customer_Name,Delivery_distance,Status
0,0.0,2023-07-01,3808,2023-07-01,2023-07-03,2023-07-04,2023-07-04,0,0,0,0,Apparel,Less Than Truckload,Low,UPS,BetaDepot,Supp-Z,ElecHouse,300,Order received today
1,0.0,2023-07-02,3808,2023-07-01,2023-07-03,2023-07-04,2023-07-04,0,0,0,0,Apparel,Less Than Truckload,Low,UPS,BetaDepot,Supp-Z,ElecHouse,300,Preparing for Shipment
2,0.0,2023-07-03,3808,2023-07-01,2023-07-03,2023-07-04,2023-07-04,2,0,0,0,Apparel,Less Than Truckload,Low,UPS,BetaDepot,Supp-Z,ElecHouse,300,Order has been shipped today
3,0.0,2023-07-04,3808,2023-07-01,2023-07-03,2023-07-04,2023-07-04,0,1,3,0,Apparel,Less Than Truckload,Low,UPS,BetaDepot,Supp-Z,ElecHouse,300,Delivered today
4,0.0,2023-07-01,487,2023-07-01,2023-07-03,2023-07-05,2023-07-05,0,0,0,0,Cosmetics & Personal Care,Less Than Truckload,Medium,FedEx,SthDock,GenEquip,MobileMax,500,Order received today
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,2023-07-07,267,2023-07-01,2023-07-03,2023-07-08,2023-07-08,0,0,0,0,Electronics,Intermodal & Rail Shipments,Low,Landstar,NWHouse,PureTech,ElegantEyes,9900,In transit
96,0.0,2023-07-08,267,2023-07-01,2023-07-03,2023-07-08,2023-07-08,0,5,7,0,Electronics,Intermodal & Rail Shipments,Low,Landstar,NWHouse,PureTech,ElegantEyes,9900,Delivered today
97,0.0,2023-07-01,376,2023-07-01,2023-07-03,2023-07-08,2023-07-08,0,0,0,0,Groceries,Air Freight,High,Maersk,BetaDepot,ClearChem,ComfortCasa,5700,Order received today
98,0.0,2023-07-02,376,2023-07-01,2023-07-03,2023-07-08,2023-07-08,0,0,0,0,Groceries,Air Freight,High,Maersk,BetaDepot,ClearChem,ComfortCasa,5700,Preparing for Shipment


In [5]:
# Step 1: Data Preprocessing
def preprocess_datetime_column(df, column_name):
    date_column = []
    if df[column_name].dtype == 'object':
        try:
            df[column_name] = pd.to_datetime(df[column_name])
            df[column_name] = (df[column_name] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')  # Convert to Unix timestamps (seconds)
            df[column_name] = df[column_name].astype('int64')
            date_column = column_name
        except ValueError:
            print(f"Column '{column_name}' cannot be converted to datetime type.")
    return df, date_column

# Step 2: Handling Categorical Data
def preprocess_categorical_column(df, column_name):
    label_encoder = LabelEncoder()
    cat_column = []
    if df[column_name].dtype == 'object':
            df[column_name] = label_encoder.fit_transform(df[column_name])
            cat_column = column_name
    return df, cat_column

def preprocess_numerical_data(df, column_name):
    ss = StandardScaler()
    df[column_name] = ss.fit_transform(df[column_name].values.reshape(-1, 1)).flatten()
    return(df)

def preprocess_data(dataset_path):
    # Load dataset
    dataset = pd.read_csv(dataset_path, sep=';')
    dataset = dataset.fillna(0)
    # Preprocess Data
    date_columns = []
    cat_columns = []
    for column in dataset.columns:
        dataset, date_column = preprocess_datetime_column(dataset, column)
        if len(date_column) > 0:
            date_columns.append(date_column)
    for column in dataset.columns:
        if column not in date_columns:
            dataset, cat_column = preprocess_categorical_column(dataset, column)
            if len(cat_column) > 0:
                cat_columns.append(cat_column)

    num_columns = list(filter(lambda x: x not in set(cat_columns), dataset.columns))
    for column in dataset.columns:
        if column in num_columns:
            dataset = preprocess_numerical_data(dataset, column)
    return dataset, cat_columns, date_columns
# Step 2: Select Columns
selected_columns = ['Days_from_order_to_shipment', 'Mode_Of_Transport', 'Carrier_name', 'Actual_Arrival_Date']


In [117]:
#preprocessing the data
dataset, cat_columns, date_columns = preprocess_data('take_home_dataset.csv')
num_columns = list(filter(lambda x: x not in set(cat_columns), dataset.columns))
focus = dataset[selected_columns]

Column 'Product_Category' cannot be converted to datetime type.
Column 'Mode_Of_Transport' cannot be converted to datetime type.
Column 'Priority' cannot be converted to datetime type.
Column 'Carrier_name' cannot be converted to datetime type.
Column 'Warehouse' cannot be converted to datetime type.
Column 'Supplier_Name' cannot be converted to datetime type.
Column 'Customer_Name' cannot be converted to datetime type.
Column 'Status' cannot be converted to datetime type.


In [7]:
dataset

,#,Date,Order_ID,Order_date,Start_Shipping_Date,Estimated_Arrival_Date,Actual_Arrival_Date,Days_from_order_to_shipment,Days_from_shipment_to_delivery,Days_from_order_to_delivery,Days_between_estimated_and_actual_arrival,Product_Category,Mode_Of_Transport,Priority,Carrier_name,Warehouse,Supplier_Name,Customer_Name,Delivery_distance,Status
0,0.0,-1.079818,0.732257,0.0,0.0,-2.021351,-1.151518,-0.403473,-0.291329,-0.331150,-0.142656,0,3,1,9,1,9,3,-0.658631,3
1,0.0,-0.811207,0.732257,0.0,0.0,-2.021351,-1.151518,-0.403473,-0.291329,-0.331150,-0.142656,0,3,1,9,1,9,3,-0.658631,4
2,0.0,-0.542595,0.732257,0.0,0.0,-2.021351,-1.151518,2.478479,-0.291329,-0.331150,-0.142656,0,3,1,9,1,9,3,-0.658631,2
3,0.0,-0.273984,0.732257,0.0,0.0,-2.021351,-1.151518,-0.403473,0.210962,0.851530,-0.142656,0,3,1,9,1,9,3,-0.658631,0
4,0.0,-1.079818,-1.152937,0.0,0.0,-1.114916,-0.929645,-0.403473,-0.291329,-0.331150,-0.142656,1,3,2,4,6,4,8,-0.589447,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.531851,-1.277822,0.0,0.0,1.604391,-0.264028,-0.403473,-0.291329,-0.331150,-0.142656,2,2,1,6,4,8,4,2.662198,1
96,0.0,0.800462,-1.277822,0.0,0.0,1.604391,-0.264028,-0.403473,2.220125,2.428436,-0.142656,2,2,1,6,4,8,4,2.662198,0
97,0.0,-1.079818,-1.215947,0.0,0.0,1.604391,-0.264028,-0.403473,-0.291329,-0.331150,-0.142656,3,0,0,7,1,2,1,1.209335,3
98,0.0,-0.811207,-1.215947,0.0,0.0,1.604391,-0.264028,-0.403473,-0.291329,-0.331150,-0.142656,3,0,0,7,1,2,1,1.209335,4


In [415]:
### this whole section is designed to get around the constraint of training on 4 columns of data. The thought process is to divide up a larger dataset to smaller column sections and evaluate chunks
### then the chunks are concatenated after similarity search and only the top results are returned.

def parse_columns(dataset):
    # Divide columns into subgroups of 4 columns each
    num_columns = len(dataset.columns)
    num_groups = (num_columns + 3) // 4  # Ceiling division to ensure all columns are covered
    column_groups = [dataset.columns[i * 4: (i + 1) * 4] for i in range(num_groups)]
    
    # Pad groups with less than 4 columns
    for i in range(num_groups):
        while len(column_groups[i]) < 4:
            column_groups[i].append(None)
    
    return column_groups

# Example usage:
larger_dataset = dataset  # Your larger dataset

# Divide columns into subgroups of 4 columns each (for the model to be able to predict)
column_groups = parse_columns(larger_dataset)

# Apply the trained model to each column group
X_test = []
for group in column_groups:
    group_inputs = []
    for column in group:
        if column is not None:
            group_inputs.append(larger_dataset[column])
        else:
            # Pad with zeros for missing columns
            group_inputs.append(np.zeros((len(larger_dataset),)))
    X_test.append(group_inputs)


In [555]:
# Define Entity Embedding Model
input_layers = []
embedding_layers = []
for column in focus.columns:
    input_layer = Input(shape=(1,))
    embedding_layer = Embedding(focus[column].nunique(), 1)(input_layer)  # Adjust embedding size as needed
    input_layers.append(input_layer)
    embedding_layers.append(embedding_layer)

concat_layer = Concatenate()(embedding_layers)
dense_layer = Dense(64, activation='relu')(concat_layer)  # Add more dense layers as needed
output_layer = Dense(1)(dense_layer)  # Adjust output layer size as needed

model = Model(inputs=input_layers, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')

# Prepare inputs and targets (unsupervised learning, so targets are the inputs themselves)
X_train = [focus[column].values for column in focus.columns]
y_train = X_train

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=1)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.8229  
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5409
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4287
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3554
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0551
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0093
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0021   
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0016   
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0018   
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0012   


In [519]:
# Extract Embeddings
embedding_model = Model(inputs=input_layers, outputs=embedding_layers)
# Initialize an empty list to store all predictions
all_predictions = []
# Loop through each index of X_test
for index_inputs in X_test:
    # Predict using the model for the current index
    predictions = embedding_model.predict(index_inputs)
    # Append the predictions to the list
    all_predictions.append(predictions)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [454]:
array = all_predictions[0]

In [549]:
all_arr = np.array(all_predictions)
all_arr = np.reshape(all_arr, (5, 4, 100))
result = all_arr[0]
count = 0
for array in range(1 ,len(all_arr)):
    count+=1
    result = np.concatenate((result, all_arr[array]), axis=0)

In [550]:
# fixing input dimensions
all_arr_T = np.transpose(all_arr, (0, 2, 1))
result_array = np.array(result)
result_array_T = result_array.transpose()

In [576]:
# Method for embedding text using Word2Vec
def text_embedding(texts, embedding_dim=4, window=5, min_count=1):
    # Tokenize texts
    tokenized_texts = [text.split() for text in texts]
    
    # Train Word2Vec model
    model = Word2Vec(sentences=tokenized_texts, vector_size=embedding_dim, window=window, min_count=min_count)
    
    # Get word vectors
    word_vectors = model.wv
    
    # Compute text embeddings
    embeddings = []
    for tokens in tokenized_texts:
        vectors = [word_vectors[word] for word in tokens if word in word_vectors]
        if vectors:
            embeddings.append(np.mean(vectors, axis=0))
        else:
            embeddings.append(np.zeros(embedding_dim))
    
    return np.array(embeddings)

# test text queries
query_texts = ["Apparel Product", "electronics", "toys"]

# Embed the text queries
query_embeddings = text_embedding(query_texts)
# Compute cosine similarity between query embeddings and dataset embeddings]
combined_sim = []
for chunk in all_arr_T:
    similarities = cosine_similarity(query_embeddings, chunk)
    combined_sim.append(similarities)

# Find indices of top-k most similar rows for each query
k = 5
top_indices = np.argsort(similarities, axis=1)[:, ::-1][:, :k]

# Function to retrieve row and column indices from a numerical value
def get_indices_from_numerical_value(numerical_value, num_cols):
    row_index = numerical_value // num_cols
    col_index = numerical_value % num_cols
    return row_index, col_index

# Display top-k most similar rows for each query
for i, query_text in enumerate(query_texts):
    print("\nTop {} most similar rows for query '{}':".format(k, query_text))

    print(top_indices[i])
    for num in top_indices[i]:

        #taking indexed numerical value from table
        numerical_value = num

        # Dimensions of the test dataframe
        num_cols = test.shape[1]

        # Retrieve row and column indices from the numerical value
        row_index, col_index = get_indices_from_numerical_value(numerical_value, num_cols)

        # Retrieve the value from the DataFrame using the indices
        cell_value = test.iloc[row_index, col_index]
        col_name = test.columns[col_index]
        # Print the value
        print("Value at row {} and column {}: {}".format(row_index, col_name, cell_value))

## ran out of time to dump to json :( sorry



Top 5 most similar rows for query 'Apparel Product':
[43 42 21 17 18]
Value at row 2 and column Order_date: 2023-07-01
Value at row 2 and column Order_ID: 3808
Value at row 1 and column Date: 2023-07-02
Value at row 0 and column Customer_Name: ElecHouse
Value at row 0 and column Delivery_distance: 300

Top 5 most similar rows for query 'electronics':
[39 41 40 14  0]
Value at row 1 and column Status: Preparing for Shipment
Value at row 2 and column Date: 2023-07-03
Value at row 2 and column #: nan
Value at row 0 and column Carrier_name: UPS
Value at row 0 and column #: nan

Top 5 most similar rows for query 'toys':
[39 14  0 82 28]
Value at row 1 and column Status: Preparing for Shipment
Value at row 0 and column Carrier_name: UPS
Value at row 0 and column #: nan
Value at row 4 and column Order_ID: 487
Value at row 1 and column Days_from_shipment_to_delivery: 0
